# Project 14: Pretraining a Tiny Transformer from Scratch

## CORE PROJECT - Understanding Base Models

## Goal
Pretrain a small transformer and watch it learn language patterns.

## Learning Objectives
- What happens during pretraining
- Next-token prediction loss
- Loss dynamics and convergence
- Text generation from trained model
- Why pretraining is expensive but powerful

## Model Configuration
```
Vocabulary: 5000-10000 tokens
Dimension: 384
Heads: 6
Layers: 4-6
Parameters: ~20-50M
Dataset: Shakespeare (~5MB)
Training time on M4: 4-12 hours
```

In [ ]:
# Setup
import torch
import torch.nn as nn
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Training on: {device}")